# Jacobi Test
Test if the jacobian works

In [1]:
# this file's version
from datetime import datetime as dt
print("Notebook version as of", dt.today().date())

Notebook version as of 2018-11-23


## Imports

In [2]:
cd ../aei

/home/wbarfuss/Drive/3.2_Beruf/1_Projects/01_DetRL/DetRL_Code/public/aei


In [3]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True)

In [4]:
cd agents/

/home/wbarfuss/Drive/3.2_Beruf/1_Projects/01_DetRL/DetRL_Code/public/aei/agents


In [5]:
#from detQ import detQ
from jacobiAEI import JacobiAEI
from jacobiQ import JacobiQ
from jacobiAC import JacobiAC
from jacobiSARSA import JacobiSARSA


In [6]:
cd ../envs/

/home/wbarfuss/Drive/3.2_Beruf/1_Projects/01_DetRL/DetRL_Code/public/aei/envs


In [7]:
from Env_2StatePD import TwoStatePD
from Env_2StateMatchingPennies import TwoStateMatchingPennies

## Parameters

In [8]:
cd ../../

/home/wbarfuss/Drive/3.2_Beruf/1_Projects/01_DetRL/DetRL_Code/public


In [9]:
Rpd = TwoStatePD().RewardTensor()
Tpd = TwoStatePD().TransitionTensor()

In [10]:
Rmp = TwoStateMatchingPennies().RewardTensor()
Tmp = TwoStateMatchingPennies().TransitionTensor()

In [11]:
def jshape(J, v=0, w=None):
    if w is None:
        w = v
    return J[:,:,v,:,:,w].reshape(4,4)

In [12]:
def vshape(X, v=0):
    return X[:,:,v].reshape(4,)

## Estimators

... for (parts of) the Jacobian tensor

In [13]:
def estJacobi(agents, X, dx):
    J = np.zeros((4, 4))
    xv = np.array(getBehaviorVaraible(X))
    
    ADDlimit = np.minimum(1-xv, dx/2) / (dx/2)
    SUBlimit = 1 -  np.maximum(-xv+dx/2, 0) / (dx/2)
    fracs = (ADDlimit - SUBlimit + 1) / 2
    print(fracs)
    
    DXvsPRE = np.array(xv) + np.eye(4) * fracs * dx
    DXvsPOS = np.array(xv) - np.eye(4) * (1-fracs) * dx
    
    for i in range(4):
        dX =\
            getBehaviorVaraible(
                 agents.TDstep(obt_behavior(*DXvsPRE[i])) -
                 agents.TDstep(obt_behavior(*DXvsPOS[i])) )

        J[:, i] = np.array(np.array(dX)) / dx
    return J

In [14]:
def est_dTss(agents, X, dx):
    dTss = np.zeros((4, 4))
    xv = np.array(getBehaviorVaraible(X))
    
    ADDlimit = np.minimum(1-xv, dx/2) / (dx/2)
    SUBlimit = 1 -  np.maximum(-xv+dx/2, 0) / (dx/2)
    fracs = (ADDlimit - SUBlimit + 1) / 2
    print(fracs)
    
    DXvsPRE = np.array(xv) + np.eye(4) * fracs * dx
    DXvsPOS = np.array(xv) - np.eye(4) * (1-fracs) * dx
    
    for i in range(4):
        DeltTss = \
             agents.obtain_Tss(obt_behavior(*DXvsPRE[i])) -\
             agents.obtain_Tss(obt_behavior(*DXvsPOS[i])) 

        dTss[:, i] = np.array(DeltTss.reshape(4)) / dx
    return dTss

In [15]:
def est_dRis(agents, X, dx):
    dRis = np.zeros((4, 4))
    xv = np.array(getBehaviorVaraible(X))
    
    ADDlimit = np.minimum(1-xv, dx/2) / (dx/2)
    SUBlimit = 1 -  np.maximum(-xv+dx/2, 0) / (dx/2)
    fracs = (ADDlimit - SUBlimit + 1) / 2
    print(fracs)
    
    DXvsPRE = np.array(xv) + np.eye(4) * fracs * dx
    DXvsPOS = np.array(xv) - np.eye(4) * (1-fracs) * dx
    
    for i in range(4):
        DeltRis =\
             agents.obtain_Ris(obt_behavior(*DXvsPRE[i])) -\
             agents.obtain_Ris(obt_behavior(*DXvsPOS[i])) 

        dRis[:, i] = np.array(DeltRis.reshape(4)) / dx
    return dRis

In [16]:
def est_dMss(agents, X, dx):
    dMss = np.zeros((4, 4))
    xv = np.array(getBehaviorVaraible(X))
    
    ADDlimit = np.minimum(1-xv, dx/2) / (dx/2)
    SUBlimit = 1 -  np.maximum(-xv+dx/2, 0) / (dx/2)
    fracs = (ADDlimit - SUBlimit + 1) / 2
    print(fracs)
    
    DXvsPRE = np.array(xv) + np.eye(4) * fracs * dx
    DXvsPOS = np.array(xv) - np.eye(4) * (1-fracs) * dx
    
    def M(agents, X):
        Tss = agents.obtain_Tss(X)
        M = np.eye(agents.Z) - agents.gamma * Tss
        return M    
    
    for i in range(4):
        dM =\
             M(agents, obt_behavior(*DXvsPRE[i])) -\
             M(agents, obt_behavior(*DXvsPOS[i])) 

        dMss[:, i] = np.array(dM.reshape(4)) / dx
    return dMss

In [17]:
def est_dinvMss(agents, X, dx):
    dinvMss = np.zeros((4, 4))
    xv = np.array(getBehaviorVaraible(X))
    
    ADDlimit = np.minimum(1-xv, dx/2) / (dx/2)
    SUBlimit = 1 -  np.maximum(-xv+dx/2, 0) / (dx/2)
    fracs = (ADDlimit - SUBlimit + 1) / 2
    print(fracs)
    
    DXvsPRE = np.array(xv) + np.eye(4) * fracs * dx
    DXvsPOS = np.array(xv) - np.eye(4) * (1-fracs) * dx
    
    def invM(agents, X):
        Tss = agents.obtain_Tss(X)
        M = np.eye(agents.Z) - agents.gamma * Tss
        return np.linalg.inv(M)    
    
    for i in range(4):
        diM =\
             invM(agents, obt_behavior(*DXvsPRE[i])) -\
             invM(agents, obt_behavior(*DXvsPOS[i])) 

        dinvMss[:, i] = np.array(diM.reshape(4)) / dx
    return dinvMss

In [18]:
def est_dVis(agents, X, dx):
    dVis = np.zeros((4, 4))
    xv = np.array(getBehaviorVaraible(X))
    
    ADDlimit = np.minimum(1-xv, dx/2) / (dx/2)
    SUBlimit = 1 -  np.maximum(-xv+dx/2, 0) / (dx/2)
    fracs = (ADDlimit - SUBlimit + 1) / 2
    print(fracs)
    
    DXvsPRE = np.array(xv) + np.eye(4) * fracs * dx
    DXvsPOS = np.array(xv) - np.eye(4) * (1-fracs) * dx
    
    for i in range(4):
        DeltVis =\
             agents.obtain_Vis(obt_behavior(*DXvsPRE[i])) -\
             agents.obtain_Vis(obt_behavior(*DXvsPOS[i])) 

        dVis[:, i] = np.array(DeltVis.reshape(4)) / dx
    return dVis

In [19]:
def est_dRisa(agents, X, dx):
    dRisa = np.zeros((4, 4))
    xv = np.array(getBehaviorVaraible(X))
    
    ADDlimit = np.minimum(1-xv, dx/2) / (dx/2)
    SUBlimit = 1 -  np.maximum(-xv+dx/2, 0) / (dx/2)
    fracs = (ADDlimit - SUBlimit + 1) / 2
    print(fracs)
    
    DXvsPRE = np.array(xv) + np.eye(4) * fracs * dx
    DXvsPOS = np.array(xv) - np.eye(4) * (1-fracs) * dx
    
    for i in range(4):
        dR =\
            getBehaviorVaraible(
                 agents.obtain_Risa(obt_behavior(*DXvsPRE[i])) -
                 agents.obtain_Risa(obt_behavior(*DXvsPOS[i])) )

        dRisa[:, i] = np.array(dR) / dx
    return dRisa

In [20]:
def est_dQisa(agents, X, dx):
    dQisa = np.zeros((4, 4))
    xv = np.array(getBehaviorVaraible(X))
    
    ADDlimit = np.minimum(1-xv, dx/2) / (dx/2)
    SUBlimit = 1 -  np.maximum(-xv+dx/2, 0) / (dx/2)
    fracs = (ADDlimit - SUBlimit + 1) / 2
    print(fracs)
    
    DXvsPRE = np.array(xv) + np.eye(4) * fracs * dx
    DXvsPOS = np.array(xv) - np.eye(4) * (1-fracs) * dx
    
    for i in range(4):
        dQ =\
            getBehaviorVaraible(
                 agents.obtain_Qisa(obt_behavior(*DXvsPRE[i])) -
                 agents.obtain_Qisa(obt_behavior(*DXvsPOS[i])) )

        dQisa[:, i] = np.array(dQ) / dx
    return dQisa

In [21]:
def est_dMaxQisa(agents, X, dx):
    dMaxQisa = np.zeros((4, 4))
    xv = np.array(getBehaviorVaraible(X))
    
    ADDlimit = np.minimum(1-xv, dx/2) / (dx/2)
    SUBlimit = 1 -  np.maximum(-xv+dx/2, 0) / (dx/2)
    fracs = (ADDlimit - SUBlimit + 1) / 2
    print(fracs)
    
    DXvsPRE = np.array(xv) + np.eye(4) * fracs * dx
    DXvsPOS = np.array(xv) - np.eye(4) * (1-fracs) * dx
    
    for i in range(4):
        dmQ =\
            getBehaviorVaraible(
                 agents.obtain_MaxQisa(obt_behavior(*DXvsPRE[i])) -
                 agents.obtain_MaxQisa(obt_behavior(*DXvsPOS[i])) )

        dMaxQisa[:, i] = np.array(dmQ) / dx
    return dMaxQisa

In [22]:
def est_dNextVis(agents, X, dx):
    dnextVis = np.zeros((4, 4))
    xv = np.array(getBehaviorVaraible(X))
    
    ADDlimit = np.minimum(1-xv, dx/2) / (dx/2)
    SUBlimit = 1 -  np.maximum(-xv+dx/2, 0) / (dx/2)
    fracs = (ADDlimit - SUBlimit + 1) / 2
    print(fracs)
    
    DXvsPRE = np.array(xv) + np.eye(4) * fracs * dx
    DXvsPOS = np.array(xv) - np.eye(4) * (1-fracs) * dx
    
    for i in range(4):
        dnVis =\
            getBehaviorVaraible(
                 agents.obtain_NextVisa(obt_behavior(*DXvsPRE[i])) -
                 agents.obtain_NextVisa(obt_behavior(*DXvsPOS[i])) )

        dnextVis[:, i] = np.array(dnVis) / dx
    return dnextVis

In [23]:
def est_dNextQisa(agents, X, dx):
    dnextQisa = np.zeros((4, 4))
    xv = np.array(getBehaviorVaraible(X))
    
    ADDlimit = np.minimum(1-xv, dx/2) / (dx/2)
    SUBlimit = 1 -  np.maximum(-xv+dx/2, 0) / (dx/2)
    fracs = (ADDlimit - SUBlimit + 1) / 2
    print(fracs)
    
    DXvsPRE = np.array(xv) + np.eye(4) * fracs * dx
    DXvsPOS = np.array(xv) - np.eye(4) * (1-fracs) * dx
    
    for i in range(4):
        dnQisa =\
            getBehaviorVaraible(
                 agents.obtain_NextQisa(obt_behavior(*DXvsPRE[i])) -
                 agents.obtain_NextQisa(obt_behavior(*DXvsPOS[i])) )

        dnextQisa[:, i] = np.array(dnQisa) / dx
    return dnextQisa

In [24]:
def est_dtTDisa(agents, X, dx):
    dtTDisa = np.zeros((4, 4))
    xv = np.array(getBehaviorVaraible(X))
    
    ADDlimit = np.minimum(1-xv, dx/2) / (dx/2)
    SUBlimit = 1 -  np.maximum(-xv+dx/2, 0) / (dx/2)
    fracs = (ADDlimit - SUBlimit + 1) / 2
    print(fracs)
    
    DXvsPRE = np.array(xv) + np.eye(4) * fracs * dx
    DXvsPOS = np.array(xv) - np.eye(4) * (1-fracs) * dx
    
    for i in range(4):
        dtTD =\
            getBehaviorVaraible(
                 agents.obtain_truncedTDisa(obt_behavior(*DXvsPRE[i])) -
                 agents.obtain_truncedTDisa(obt_behavior(*DXvsPOS[i])) )

        dtTDisa[:, i] = np.array(dtTD) / dx
    return dtTDisa

In [25]:
def est_dAisa(agents, X, dx):
    dAisa = np.zeros((4, 4))
    xv = np.array(getBehaviorVaraible(X))
    
    ADDlimit = np.minimum(1-xv, dx/2) / (dx/2)
    SUBlimit = 1 -  np.maximum(-xv+dx/2, 0) / (dx/2)
    fracs = (ADDlimit - SUBlimit + 1) / 2
    print(fracs)
    
    DXvsPRE = np.array(xv) + np.eye(4) * fracs * dx
    DXvsPOS = np.array(xv) - np.eye(4) * (1-fracs) * dx
    
    for i in range(4):
        dA =\
            getBehaviorVaraible(
                 agents.obtain_A(obt_behavior(*DXvsPRE[i])) -
                 agents.obtain_A(obt_behavior(*DXvsPOS[i])) )

        dAisa[:, i] = np.array(dA) / dx
    return dAisa

## Test

In [26]:
def obt_behavior(X000, X010, X100, X110):
    Xinit = np.zeros((2,2,2))
    Xinit[0, 0, 0] = X000
    Xinit[1, 0, 0] = X100
    Xinit[0, 1, 0] = X010
    Xinit[1, 1, 0] = X110

    Xinit[0, 0, 1] = 1.0 - Xinit[0, 0, 0]
    Xinit[1, 0, 1] = 1.0 - Xinit[1, 0, 0]
    Xinit[0, 1, 1] = 1.0 - Xinit[0, 1, 0]
    Xinit[1, 1, 1] = 1.0 - Xinit[1, 1, 0]
    
    return Xinit

In [27]:
def getBehaviorVaraible(X):
    return X[0, 0, 0], X[0, 1, 0], X[1, 0, 0], X[1, 1, 0]

In [28]:
alpha = 0.1
beta = 4.0
gamma = 0.23

In [29]:
agS = JacobiSARSA(Tpd, Rpd, alpha, beta, gamma)
agA = JacobiAC(Tmp, Rmp, alpha, beta, gamma)
#agApd = JacobiAC(Tpd, Rpd, alpha, beta, gamma)
agQ = JacobiQ(Tmp, Rmp, alpha, beta, gamma)

In [30]:
X = obt_behavior(X000=0., X010=0.1, X100=0., X110=0.)
X0 = obt_behavior(X000=0., X010=0., X100=0., X110=0.)
X1 = obt_behavior(X000=1., X010=1., X100=1., X110=1.)
X01 = obt_behavior(X000=0., X010=1., X100=0., X110=0.)

X5 = obt_behavior(X000=0.5, X010=0.5, X100=0.5, X110=0.5)

In [31]:
# Choose one of the following:

agents = agA
# agents = agQ
# agents = agS

### fULL Jacobi--

In [32]:
X = agents.random_behavior()

In [33]:
estJacobi(agents, X, 0.00001).round(3)

[ 0.5  0.5  0.5  0.5]


array([[ 1.091,  0.008,  0.132, -0.   ],
       [-0.012,  1.001,  0.01 , -0.153],
       [-0.087,  0.   ,  0.904,  0.   ],
       [ 0.   ,  0.115,  0.   ,  1.002]])

In [34]:
J = agents.jacobi(X)
jshape(J, v=1, w=1).round(3)

array([[ 1.091,  0.008,  0.132, -0.   ],
       [-0.012,  1.001,  0.01 , -0.153],
       [-0.087, -0.   ,  0.904,  0.   ],
       [-0.   ,  0.115,  0.   ,  1.002]])

### dA

In [35]:
est_dAisa(agents, X, 0.000001).round(3)

[ 0.5  0.5  0.5  0.5]


array([[ 1.099,  0.005,  0.087, -0.   ],
       [-0.013,  1.098,  0.011, -0.169],
       [-0.269,  0.   ,  1.083,  0.   ],
       [ 0.   ,  0.077,  0.   ,  1.002]])

In [36]:
jshape(agents.obtain_dA(X)).round(3)

array([[ 1.107,  0.012,  0.081, -0.   ],
       [ 0.004,  1.111, -0.003, -0.17 ],
       [-0.318, -0.011,  1.114,  0.   ],
       [-0.008,  0.068,  0.006,  1.002]])

### dQisa

In [37]:
X = agents.random_behavior()

In [38]:
jshape(agents.obtain_dQisa(X))

array([[-0.01002841, -0.0545599 ,  0.76422097,  0.03452693],
       [-0.02864048, -0.02831465, -0.0068941 , -0.74323504],
       [-0.75997159,  0.0545599 ,  0.00577903, -0.03452693],
       [ 0.02864048,  0.79831465,  0.0068941 , -0.02676496]])

In [39]:
est_dQisa(agents, X, 0.000001)

[ 0.5  0.5  0.5  0.5]


array([[-0.01002841, -0.0545599 ,  0.76422097,  0.03452693],
       [-0.02864048, -0.02831465, -0.0068941 , -0.74323504],
       [-0.75997159,  0.0545599 ,  0.00577903, -0.03452693],
       [ 0.02864048,  0.79831465,  0.0068941 , -0.02676496]])

### dRisa

In [40]:
X = agents.random_behavior()

In [41]:
jshape(agents.obtain_dRisa(X))

array([[ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0., -1.],
       [-1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.]])

In [42]:
est_dRisa(agents, X, 0.0000000001).round(4)

[ 0.5  0.5  0.5  0.5]


array([[ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0., -1.],
       [-1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.]])

### dRis

In [43]:
vshape(X)

array([ 0.52756771,  0.26542263,  0.71948686,  0.41125061])

In [44]:
agents.obtain_dRis(X)[:,:,:,:,0].reshape(4,4).round(4)

array([[ 0.439 ,  0.    ,  0.0551,  0.    ],
       [ 0.    ,  0.1775,  0.    ,  0.4692],
       [-0.439 ,  0.    , -0.0551,  0.    ],
       [ 0.    , -0.1775,  0.    , -0.4692]])

In [45]:
est_dRis(agents, X, 0.00001).round(4)

[ 0.5  0.5  0.5  0.5]


array([[ 0.439 ,  0.    ,  0.0551,  0.    ],
       [ 0.    ,  0.1775,  0.    ,  0.4692],
       [-0.439 ,  0.    , -0.0551,  0.    ],
       [ 0.    , -0.1775,  0.    , -0.4692]])

### dTss

In [46]:
vshape(X)

array([ 0.52756771,  0.26542263,  0.71948686,  0.41125061])

In [47]:
agents.obtain_dTss(X)[:,:,:,:,0].reshape(4,4).round(4)

array([[-1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.]])

In [48]:
est_dTss(agents, X, 0.00001).round(4)

[ 0.5  0.5  0.5  0.5]


array([[-1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.]])

### dMss

In [49]:
(agents.gamma * agents.obtain_dTss(X)[:,:,:,:,0].reshape(4,4)).round(4)

array([[-0.23,  0.  ,  0.  ,  0.  ],
       [ 0.23,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.23,  0.  ,  0.  ],
       [ 0.  ,  0.23,  0.  ,  0.  ]])

In [50]:
est_dMss(agents, X, 0.00001).round(4)

[ 0.5  0.5  0.5  0.5]


array([[ 0.23,  0.  ,  0.  ,  0.  ],
       [-0.23,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.23,  0.  ,  0.  ],
       [ 0.  , -0.23,  0.  ,  0.  ]])

### dinvMss

In [51]:
vshape(X)

array([ 0.52756771,  0.26542263,  0.71948686,  0.41125061])

In [52]:
Tss = agents.obtain_Tss(X)
Ris = agents.obtain_Ris(X)
dTss = agents.obtain_dTss(X)
dRis = agents.obtain_dRis(X)
M = np.eye(agents.Z) - agents.gamma*Tss
dM = - agents.gamma * dTss
invM = np.linalg.inv(M)
dinvM = np.tensordot(np.tensordot(-1*invM, dM, axes=[1, 0]),
                     invM, axes=[1, 0])

dinvM = np.tensordot(-1*invM,
                     np.tensordot( dM, invM, axes=[1, 0]),
                     axes=[1, 0])

dinvM2 = \
np.einsum(-1 * invM, [0, 1],
          np.einsum(dM, [0, 1, 2, 3, 4],
                    invM, [1, 5], [0, 5, 2, 3, 4]), [1, 2, 3, 4, 5],
         [0, 2, 3, 4, 5]
         )


dinvM2[:,:,:,:,0].reshape(4,4).round(4)

# np.transpose(dinvM, axes=[1, 0, 2, 3, 4])[:,:,:,:,0].reshape(4,4).round(4)
            

#dinvM[0, 1, 0, 1, 0]

array([[-0.2495, -0.0322,  0.    ,  0.    ],
       [ 0.2495,  0.0322,  0.    ,  0.    ],
       [-0.0449, -0.2368,  0.    ,  0.    ],
       [ 0.0449,  0.2368,  0.    ,  0.    ]])

In [53]:
est_dinvMss(agents, X, 0.0001)

[ 0.5  0.5  0.5  0.5]


array([[-0.249476  , -0.03223971,  0.        ,  0.        ],
       [ 0.249476  ,  0.03223971,  0.        ,  0.        ],
       [-0.04489008, -0.23682563,  0.        ,  0.        ],
       [ 0.04489008,  0.23682563,  0.        ,  0.        ]])

### dVis

In [54]:
agents.obtain_dVis(X)[:,:,:,:,0].reshape(4,4).round(4)

array([[ 0.3784,  0.019 ,  0.0488,  0.0537],
       [ 0.0681,  0.1394,  0.0088,  0.3944],
       [-0.3784, -0.019 , -0.0488, -0.0537],
       [-0.0681, -0.1394, -0.0088, -0.3944]])

In [55]:
est_dVis(agents, X, 0.0000001).round(4)

[ 0.5  0.5  0.5  0.5]


array([[ 0.3784,  0.019 ,  0.0488,  0.0537],
       [ 0.0681,  0.1394,  0.0088,  0.3944],
       [-0.3784, -0.019 , -0.0488, -0.0537],
       [-0.0681, -0.1394, -0.0088, -0.3944]])

The subsequent parts of the TD error are agent specific:

### dtruncedTD

In [56]:
agents = agQ

In [57]:
est_dtTDisa(agents, X, 0.000001).round(4)

[ 0.5  0.5  0.5  0.5]


array([[ 0.0157,  0.0006,  0.772 , -0.1695],
       [ 0.0157,  0.0006,  0.002 , -0.9395],
       [-0.6629, -0.0958, -0.0018, -0.0099],
       [ 0.1191,  0.7557, -0.0016, -0.0111]])

In [58]:
jshape(agents.obtain_dtruncedTDisa(X)).round(4)

array([[ 0.0157,  0.0006,  0.772 , -0.1695],
       [ 0.0157,  0.0006,  0.002 , -0.9395],
       [-0.6629, -0.0958, -0.0018, -0.0099],
       [ 0.1191,  0.7557, -0.0016, -0.0111]])

In [59]:
(1-gamma) * jshape(agents.obtain_dRisa(X))

array([[ 0.  ,  0.  ,  0.77,  0.  ],
       [ 0.  ,  0.  ,  0.  , -0.77],
       [-0.77,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.77,  0.  ,  0.  ]])

### d max Qisa

In [60]:
X = agents.random_behavior()

In [61]:
jshape(agents.obtain_dmaxQisa(X))

array([[-0.00491781, -0.06297986,  0.00293498,  0.60466866],
       [-0.00491781, -0.06297986,  0.00293498,  0.60466866],
       [ 0.62431115,  0.48099451, -0.00641   ,  0.14206622],
       [ 0.08300145,  1.02306867, -0.00370803,  0.1601558 ]])

In [62]:
est_dMaxQisa(agents, X, 0.00001)

[ 0.5  0.5  0.5  0.5]


array([[-0.00491781, -0.06297986,  0.00293498,  0.60466866],
       [-0.00491781, -0.06297986,  0.00293498,  0.60466866],
       [ 0.62431115,  0.48099451, -0.00641   ,  0.14206622],
       [ 0.08300145,  1.02306867, -0.00370803,  0.1601558 ]])

### dnext Qisa

In [63]:
agents = agS

In [64]:
X = agents.random_behavior()

In [65]:
est_dNextQisa(agents, X, 0.0001)

[ 0.5  0.5  0.5  0.5]


array([[-2.73676378, -0.72043079,  6.77476298,  2.98255644],
       [-3.21101633, -0.56170064,  4.60360859, -0.52566693],
       [-1.0776136 ,  5.95712592, -0.34536763, -2.64196661],
       [ 2.4433501 ,  5.75409548, -0.84200585, -1.62953469]])

In [66]:
jshape(agents.obtain_dnextQisa(X))

array([[-2.73676378, -0.72043079,  6.77476298,  2.98255644],
       [-3.21101633, -0.56170064,  4.60360858, -0.52566693],
       [-1.0776136 ,  5.95712592, -0.34536763, -2.64196661],
       [ 2.4433501 ,  5.75409548, -0.84200585, -1.62953469]])

### d next Visa

In [67]:
agents = agA

In [68]:
# X = X1
X = agents.random_behavior()

In [69]:
jshape(agents.obtain_dnextVisa(X))

array([[-0.01153777, -0.21626885,  0.02191929,  0.03687991],
       [-0.01153777, -0.21626885,  0.02191929,  0.03687991],
       [ 0.03057025,  0.14167422, -0.07913182, -0.02415943],
       [ 0.05016424,  0.10950945, -0.09530126, -0.02056436]])

In [70]:
est_dNextVis(agents, X, 0.00001)

[ 0.5  0.5  0.5  0.5]


array([[-0.01153777, -0.21626885,  0.02191929,  0.03687991],
       [-0.01153777, -0.21626885,  0.02191929,  0.03687991],
       [ 0.03057025,  0.14167422, -0.07913182, -0.02415943],
       [ 0.05016424,  0.10950945, -0.09530126, -0.02056436]])